# Objet : Test opendata des données de qualité de l'air

## Objectif

- valider sur des cas réels l'outil de traitement des "listes indexées"
- identifier les apports que pourraient avoir ce type d'outil

## Résultats
- à compléter

## Usages possibles 
- à compléter

## Autres points
- chargement sur MongoDB à tester
- Outil de requète à tester

données utilisées : https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/temps-reel/2022/

------
## Initialisation
- lecture des fichiers de 01/2022 issus de l'api (un fichier par jour)

In [3]:
from pprint import pprint
from time import time
from datetime import datetime
import csv
from observation import Ilist, Iindex, util, Observation, ESSearch
from copy import copy
import pandas as pd
from test_mongo import clientMongo

chemin = 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/air/data_lcsqa/'
collec = clientMongo()['air_quality']['lcsqa_1']

In [2]:
envoi = False
nb_fichiers = 1
annee = 2022
mois = 1
jour = 1
# chargement des fichiers du mois et envoi dans mongoDB
for i in range(nb_fichiers):
    file = chemin + 'FR_E2_' + str(annee) + '-' + format(mois, '02d') +'-' + format(jour + i, '02d') +'.csv'
    data = pd.read_csv(file, sep=';'))
    data[['Date de début','Date de fin']] = data2[['Date de début','Date de fin']].astype('datetime64')
    data = data.astype('category')
    data.pop('valeur brute')
    print('data : \n', len(data), '\n', list(data), '\n')
    if envoi:
        name = 'air quality {:4d}-{:02d}-{:02d}'.format(annee, mois, jour + i)
        json_lcsqa = Observation(Ilist.obj(data), name=name).json(modecodec='dict')
        print(collec.insert_one(json_lcsqa).inserted_id)

data2 : 
 49392 
 ['Date de début', 'Date de fin', 'Organisme', 'code zas', 'Zas', 'code site', 'nom site', "type d'implantation", 'Polluant', "type d'influence", 'discriminant', 'Réglementaire', "type d'évaluation", 'procédure de mesure', 'type de valeur', 'valeur', 'unité de mesure', 'taux de saisie', 'couverture temporelle', 'couverture de données', 'code qualité', 'validité'] 
 9.283959865570068


In [12]:
# définition des critères de recherche
periode_debut = ['Date de début', datetime(2022, 1, 1, 10), '>='   ]
periode_fin   = ['Date de fin',   datetime(2022, 1, 1, 15), '<='   ]
lieu          = ['nom site',      'MARSEILLE',              'regex']
polluant      = ['Polluant',      'PM',                     'regex']

t0 = time()
result = ESSearch(collec, []).execute()
#result = ESSearch(collec, [periode_fin]).execute()
#result = ESSearch(collec, [periode_debut, periode_fin]).execute()
#result = ESSearch(collec, [periode_debut, periode_fin, lieu, polluant]).execute()
print(result, time()-t0)

TypeError: argument of type 'datetime.datetime' is not iterable

In [5]:
print(result[1])

IndexError: list index out of range